In [37]:
#Se importan las librerias necesarias 
import array
import random
import json

import numpy

from deap import algorithms
from deap import base
from deap import creator
from deap import tools

In [38]:
with open("gr17.json", "r") as tsp_data: #En esta parte se lee el documento 
    tsp = json.load(tsp_data)

distance_map = tsp["DistanceMatrix"] #Se obtiene la matriz de distancias 

IND_SIZE = tsp["TourSize"] #Se obtiene el número de nodos 


creator.create("FitnessMin", base.Fitness, weights=(-1.0,)) #Se crea la propiedad fitness para minimizar el coste 
creator.create("Individual", array.array, typecode='i', fitness=creator.FitnessMin) #Se crea una individuo

toolbox = base.Toolbox() #Inicialización de la caja de herramientas 

toolbox.register("indices", random.sample, range(IND_SIZE), IND_SIZE)#Función que crea un vector aleatorio con los índices

toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.indices) #Se le asigna al individuo un genoma aleatorio
toolbox.register("population", tools.initRepeat, list, toolbox.individual) #Se crea la población 

def evalTSP(individual): #Esta función recorre la matriz tomando muestras 
    distance = distance_map[individual[-1]][individual[0]]
    for gene1, gene2 in zip(individual[0:-1], individual[1:]):
        distance += distance_map[gene1][gene2]
    return distance,

toolbox.register("mate", tools.cxPartialyMatched) #Probabilidad de reproducción 
toolbox.register("mutate", tools.mutShuffleIndexes, indpb=0.05) #Probabilidad de mutación 
toolbox.register("select", tools.selTournament, tournsize=3) #Número de casos a ser seleccionados 
toolbox.register("evaluate", evalTSP) #Obtiene la información de la matriz 

def main():
    random.seed(169) #Se crea una semilla aleatoria 

    pop = toolbox.population(n=100) #Se especifica el número de población 

    hof = tools.HallOfFame(1) #Se especifica cuántas rutas obtener 
    stats = tools.Statistics(lambda ind: ind.fitness.values) #Registar las estadísticas 
    stats.register("avg", numpy.mean) #Promedio 
    stats.register("std", numpy.std) #Desviación estándar 
    stats.register("min", numpy.min) #Valor mínimo 
    stats.register("max", numpy.max) #Valor máximo 
    
    algorithms.eaSimple(pop, toolbox, 0.7, 0.2, 500, stats=stats, 
                        halloffame=hof)
    
    return pop, stats, hof

if __name__ == "__main__":
    pop,stats,hof=main()
    print(hof)
    print(evalTSP(hof[0]))

gen	nevals	avg    	std    	min 	max 
0  	100   	4683.27	424.544	3433	5516
1  	77    	4365.52	500.838	3263	5630
2  	73    	4168.74	545.733	3263	5609
3  	78    	4020   	483.167	3101	5209
4  	65    	3811.78	459.417	3101	5448
5  	64    	3644.23	394.816	3081	4910
6  	81    	3539.39	356.106	2931	5026
7  	72    	3410.62	303.904	2853	4591
8  	76    	3335.31	435.934	2677	5003
9  	72    	3140.74	291.316	2677	4295
10 	74    	2973.91	171.015	2667	3537
11 	76    	2901.79	239.243	2585	4189
12 	75    	2825.52	182.245	2585	3841
13 	78    	2776.66	159.559	2507	3487
14 	77    	2741.8 	174.197	2506	3593
15 	68    	2725.18	266.157	2480	4402
16 	74    	2643.69	152.127	2451	3244
17 	71    	2685.63	297.106	2449	4330
18 	74    	2648.87	294.304	2404	3819
19 	71    	2678.11	338.3  	2380	3936
20 	75    	2609.94	239.076	2380	3734
21 	76    	2610.43	337.335	2380	4471
22 	77    	2523.7 	266.501	2327	3809
23 	77    	2500.29	279.056	2327	3932
24 	89    	2499.72	333.523	2327	4068
25 	73    	2434.73	245.115	2246	3588
2

In [43]:
def main():
    random.seed(64)
    NGEN = 600 #Número de generaciones 
    MU = 10 #Número de individuos para la siguiente generación 
    LAMBDA = 23 #Número de hijos en cada generación 
    CXPB = 0.7 #Probabilidad de descendencia 
    MUTPB = 0.2 #Probabilidad de mutación 
    
    pop = toolbox.population(n=MU)#Se especifica la población 
    hof = tools.ParetoFront()
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", numpy.mean, axis=0)
    stats.register("std", numpy.std, axis=0)
    stats.register("min", numpy.min, axis=0)
    stats.register("max", numpy.max, axis=0)
    
    algorithms.eaMuPlusLambda(pop, toolbox, MU, LAMBDA, CXPB, MUTPB, NGEN, stats=stats,
                              halloffame=hof)
    
    return pop, stats, hof
                 
if __name__ == "__main__":
    pop,stats,hof=main()
    print(hof)
    print(evalTSP(hof[0]))

gen	nevals	avg      	std           	min    	max    
0  	100   	[4683.37]	[459.69769751]	[3219.]	[5621.]
1  	20    	[4262.7] 	[340.63178067]	[3738.]	[4862.]
2  	20    	[3966.4] 	[324.68144388]	[3328.]	[4597.]
3  	20    	[3769.1] 	[170.88853092]	[3328.]	[3936.]
4  	22    	[3738.]  	[0.]          	[3738.]	[3738.]
5  	22    	[3736.3] 	[5.1]         	[3721.]	[3738.]
6  	22    	[3732.9] 	[7.79037868]  	[3721.]	[3738.]
7  	23    	[3726.1] 	[7.79037868]  	[3721.]	[3738.]
8  	23    	[3722.7] 	[5.1]         	[3721.]	[3738.]
9  	20    	[3721.]  	[0.]          	[3721.]	[3721.]
10 	20    	[3721.]  	[0.]          	[3721.]	[3721.]
11 	20    	[3721.]  	[0.]          	[3721.]	[3721.]
12 	22    	[3721.]  	[0.]          	[3721.]	[3721.]
13 	20    	[3714.8] 	[18.6]        	[3659.]	[3721.]
14 	21    	[3686.5] 	[84.87078414] 	[3438.]	[3721.]
15 	22    	[3607.8] 	[138.64111944]	[3438.]	[3721.]
16 	22    	[3551.2] 	[138.64111944]	[3438.]	[3721.]
17 	17    	[3466.3] 	[84.9]        	[3438.]	[3721.]
18 	19    	[

In [44]:
def main():
    random.seed()
    MU, LAMBDA = 25, 100
    pop = toolbox.population(n=MU)
    hof = tools.HallOfFame(1)
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", numpy.mean)
    stats.register("std", numpy.std)
    stats.register("min", numpy.min)
    stats.register("max", numpy.max)
    
    pop, logbook = algorithms.eaMuCommaLambda(pop, toolbox, mu=MU, lambda_=LAMBDA, 
        cxpb=0.6, mutpb=0.3, ngen=500, stats=stats, halloffame=hof)
    
    return pop, logbook, hof

if __name__ == "__main__":
    pop,stats,hof=main()
    print(hof)
    print(evalTSP(hof[0]))

gen	nevals	avg    	std    	min 	max 
0  	25    	4824.56	321.312	4355	5564
1  	83    	4527.88	172.87 	4220	4768
2  	86    	4294.4 	217.553	3926	4617
3  	93    	4096.36	252.73 	3502	4605
4  	88    	3953.84	238.259	3653	4562
5  	90    	3852.32	253.713	3452	4447
6  	89    	3931.76	331.165	3452	4474
7  	85    	3784.88	237.195	3452	4378
8  	89    	3714.32	213.15 	3213	4270
9  	87    	3599.16	195.478	3213	4124
10 	93    	3530.04	209.545	3213	4279
11 	87    	3541.72	247.638	3249	4126
12 	91    	3547.12	246.323	3249	4113
13 	89    	3471.6 	237.911	3249	3962
14 	86    	3466.8 	266.221	3067	4014
15 	83    	3320.76	121.511	3249	3671
16 	90    	3284.88	139.19 	3187	3916
17 	92    	3226.4 	52.9921	3005	3281
18 	93    	3189.36	69.7079	2976	3249
19 	92    	3133.36	90.8766	2966	3249
20 	90    	3079.96	97.0396	2966	3249
21 	89    	2994.48	98.3114	2741	3257
22 	85    	2936.08	95.6556	2741	3179
23 	89    	2856.92	98.1549	2741	3057
24 	94    	2762.12	32.5494	2722	2833
25 	89    	2731.44	41.7977	2644	2833
2